In [6]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.models import Model
from keras.layers import Input
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D

from keras.models import Sequential, ask_to_proceed_with_overwrite
from keras.layers import Dense
from keras.layers.core import Activation, Dense
from keras.layers import SimpleRNN, LSTM,  GRU,SeparableConvolution2D
from keras.callbacks import ModelCheckpoint,EarlyStopping

from keras.layers.wrappers import TimeDistributed

import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.core import Activation, Dense
import tensorflow as tf
import glob

import scipy
import numpy as np
from PIL import Image
import random
import pandas as pd
import random

In [79]:

def getModel2( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    #vgg_model = VGG16( weights='imagenet', include_top=False )
    vgg_model = VGG16(weights='imagenet', include_top=False)

#     for layer in vgg_model.layers:
#         layer.trainable = False

    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    predictions = Dense(output_dim ,activation='softmax')(x)

# this is the model we will train
    model = Model(input=vgg_model.input, output=predictions)

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate

    return model    

def load_model(weights_path, output_dim):

    model = getModel2( output_dim ) 
    for k,layer in model.layers_by_depth.items()[:]:
        layer[0].trainable = False
#     for layer in model.layers:
#         layer.trainable = False
        
    model.load_weights(weights_path)
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_crossentropy'])   
    return model

def load_rnn_model(weights):
    model_rnn3=Sequential()

    model_rnn3.add(TimeDistributed(trained_model, input_shape=(max_frames,224,224,3)))
    model_rnn3.add(GRU(output_dim=100,return_sequences=True))
    model_rnn3.add(GRU(output_dim=50,return_sequences=False))
    model_rnn3.add(Dropout(.2))
    model_rnn3.add(Dense(number_of_brands,activation='softmax'))
    model_rnn3.load_weights(weights)

    model_rnn3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy', 'categorical_crossentropy'])

    return model_rnn3

labels_dict = {'apple': 2, 'cocacola': 0, 'nike': 4, 'pepsi': 1, 'starbucks': 3}
reversed_dict = dict([(x[1],x[0]) for x in labels_dict.items()])

def get_brand(softmaxes):
    v = np.max(softmaxes.mean(axis=0))
    if v > 0.6:
        return reversed_dict[np.argmax(softmaxes.mean(axis=0))]
    return "Noise"


In [8]:
trained_model = load_model("weights-improvement-VGG16-LOGO5-04-0.9940.hdf5", 5)

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [11]:
max_frames = 5
number_of_brands = 5
rnn_model = load_rnn_model("RNN-05-0.8704.hdf5")

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:47: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=100, return_sequences=True)`
/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:48: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(units=50, return_sequences=False)`


In [12]:
## Download Video

In [80]:
url = "https://www.youtube.com/watch?v=uc9Bi52ZQ7A"

In [81]:
from pytube import YouTube
yt = YouTube(url)

In [82]:
video = yt.get('mp4', '720p')

In [83]:
cmd1 = "rm /tmp/tmp_1"
sp.call(cmd1,shell=True)
video.download('/tmp/tmp_1')

In [84]:
## Format video

In [85]:
import subprocess as sp
import sys

In [86]:
filename = "/tmp/tmp_1"

cmd1 = "rm -r /tmp/tmp_1_1"
sp.call(cmd1,shell=True)

cmd1 = "mkdir /tmp/tmp_1_1"
sp.call(cmd1,shell=True)
outputpath= "/tmp/tmp_1_1/"
        
outputfile = outputpath + "f"
cmd2='ffmpeg -i '+filename+' -vframes 100  -r 2 -s 224x224 ' + outputfile + '_%04d.jpg'
sp.call(cmd2,shell=True)
sys.stdout.write("Saved it at " + outputfile +"\n")

Saved it at /tmp/tmp_1_1/f


In [87]:
## Get top sliding window

In [88]:
import glob

In [89]:
frames = glob.glob("/tmp/tmp_1_1/*")
x_mean = 0.39533365588770686

In [90]:
imgs = []
for frame in frames:
    img = np.asarray(Image.open(frame))/255.
    imgs.append(img)
    
imgs = np.array(imgs) - x_mean
    
    
    
    

In [91]:
softmaxes = trained_model.predict(imgs)

In [76]:
avgs = []
for i in range(0, len(softmaxes)-max_frames ):
    avgs.append(softmaxes[i:i+max_frames].mean(axis=0))
    
    
avgs = np.array(avgs)


In [77]:
np.argmax(avgs, axis=1)

array([2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2,
       2, 0, 0])

In [75]:
get_brand(softmaxes)

0.572072


'apple'